<a href="https://colab.research.google.com/github/akshayrajeev1996/Object-Detection-on-Colaboratory/blob/master/DSPLabProject_Object_Detection_on_video_using_Google_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This implementation of the notebook will use [Darknet](https://pjreddie.com/darknet/) and the [YOLOv3 algorithm](https://arxiv.org/abs/1804.02767) to implement object detection on a video input using Google Colaboratory. Additionally, we shall use a [PyTorch implementation](https://github.com/ayooshkathuria/pytorch-yolo-v3) of the same algorithm and compare results to the original implementation of the network. This notebook will cover the following:

* Configuring notebook to run the project.
  *   Enabling GPU computation on the notebook.
  *   Checking CUDA installation on the VM and installing cudNN library.
  * Mounting Google drive files on the notebook to use directories uploaded to your google drive. (Note: This is a 2 step authentication process from Google. The directory in use will be shared via shareable link. Add to personal drive and authenticate the drive for running this notebook.)
* Download and setup of the two repositories
  * Compiling darknet on the local VM. We will be using the darknet implementation by [AlexeyAB](https://github.com/AlexeyAB/darknet).
  * Downloading the PyTorch implementation repository. We will use a forked version of the original that have added additional configurations enabled specifically for this project. Since Google Colaboratory has all the requirements pre-installed we will not require a "requirements.txt" file.
  * Download pre-trained weights on the [COCO](http://cocodataset.org/) dataset.
* Running inference 
  * Download a video file of choice. We've added a list of public test copyright free videos that you are allowed to use. This [gistfile](https://gist.github.com/jsturgis/3b19447b304616f18657) contains the description.
  *   Running detection on the video file using pre-trained weights. 
*  Copying the output file to mounted drive for viewing the file. **Please note**: Google Colaboratory doesn't support video widgets as the notebook is spawned with a low input-output data rate. 





# **Configuring the notebook for the project**


**Step 0**: Enabling GPU runtime for hardware acceleration.
Go to **Runtime > Change runtime type > Hardware Accelerator**. Change the value from **None** to **GPU** from drop down menu.

### **Connecting to Google Drive from the notebook**

**Step 1**: Add [this folder](https://drive.google.com/open?id=1KljEBc8prgqR4wo2ujZVnuUm4mtLoTGh) to your NYU id google drive. The given folder has the structure:
```
DSP-Project
│   cudnn-10.0-linux-x64-v7.5.0.56.tgz
└───Demo-Videos
│   │   pyTorch_video.mp4
│   │   darknet_video.mp4

```
We require this step to enable cudNN libraries on this virtual machine. The file [cudnn-10.0-linux-x64-v7.5.0.56.tgz](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz) is owned by Nvidia and otherwise requires developer login to download. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### **Checking CUDA release version and installing cudNN based on version of CUDA available**

In [0]:
#Check CUDA driver pre-installed on the machine
!/usr/local/cuda/bin/nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


As the version present is 10.0 pre-installed on the machine, we will download the [CUDNN version](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz) for 10.0 available on the NVIDIA website.
This file is then uploaded to the google drive folder named **DSP-Project** maintained for this implementation. We shall unzip the file on the local VM and enable permissions to use this library.


In [0]:
!tar -xvzf gdrive/My\ Drive/DSP-Project/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local
!chmod a+r /usr/local/cuda/include/cudnn.h

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a


# **Setting up repositories**

### **Compiling Darknet**
As this project was originally written in C, we will have to make the project in this machine. Since this is a GPU enabled machine, we shall modify the makefile to incorporate the necessary changes. 

In [0]:
#AlexeyAB repository
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

#Make changes to makefile
import fileinput  
STRINGS_TO_REPLACE = [
    ('GPU=0', 'GPU=1'), 
    ('OPENCV=0', 'OPENCV=1')
]

for line in fileinput.input('./Makefile', inplace=True):
    replaced = False
    for search, replace in STRINGS_TO_REPLACE:
        if search in line:
            print(line.rstrip().replace(search, replace))
            replaced = True
            break
    if not replaced:
        print(line.rstrip())
fileinput.close()
!make


### **Downloading the python implementation of the algorithm**

In [0]:
%cd .. 
!git clone https://github.com/akshayrajeev1996/pytorch-yolo-v3.git

### **Download pre-trained weights**

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights
!cp yolov3.weights ./darknet/
!cp yolov3.weights ./pytorch-yolo-v3/

#### **Sanity Check:**
Lets take a quick look at the current state of the machine. Our present directory tree should like this. 
```
content
│   yolov3.weights
└───darknet
│   │   ...
└───gdrive
│   │   My Drive
└───pytorch-yolo-v3
│   │   ...
└───sample-data
│   │   ...

``` 

In [0]:
#To confirm the directory tree run this code block
!ls -la 

# **Running Inference**


### **List of public test videos**


In [0]:
videos=[{"title":"ForBiggerFun.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerFun.mp4"},
        {"title":"ForBiggerBlazes.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4"},
        {"title":"ForBiggerEscapes.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerEscapes.mp4"},
        {"title":"ForBiggerJoyrides.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerJoyrides.mp4"},
        {"title":"ForBiggerMeltdowns.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerMeltdowns.mp4"},
        {"title":"Sintel.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/Sintel.mp4"},
        {"title":"SubaruOutbackOnStreetAndDirt.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/SubaruOutbackOnStreetAndDirt.mp4"},
        {"title":"TearsOfSteel.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/TearsOfSteel.mp4"},
        {"title":"WeAreGoingOnBullrun.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/WeAreGoingOnBullrun.mp4"},
        {"title":"WhatCarCanYouGetForAGrand.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/WhatCarCanYouGetForAGrand.mp4"},
        {"title":"VolkswagenGTIReview.mp4","url":"http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/VolkswagenGTIReview.mp4"}]

### **Download the video**
Change the list index from **0->10** to download the video file of choice. We shall be using the video at index 0 as it is a video of length 1 minute with a lot of objects to detect. 

In [0]:
def download_video(vid):
  title=vid["title"]
  url=vid["url"]
  !wget $url
  !cp $title ./pytorch-yolo-v3/
  !cp $title ./darknet
  
### CHANGE INDEX AT NEXT LINE TO CHOOSE A DIFFERENT VIDEO
video_file=videos[0]
download_video(video_file)

### **Running both detectors**


In [0]:
def run_detector(video_file):
  from imutils.video import count_frames
  import time
  total_frames=count_frames(video_file)
  %cd darknet
  start=time.time()
  !./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights $video_file -dont_show -out_filename darknet_video.mp4
  avg_fps_darknet=total_frames/(time.time()-start)
  %cd ../pytorch-yolo-v3
  start=time.time()
  !python video_demo.py --video $video_file --dont_show 1
  avg_fps_python=total_frames/(time.time()-start)
  %cd ..
  return avg_fps_darknet,avg_fps_python

In [0]:
# Run the detection function
avg_darknet,avg_python= run_detector(video_file['title'])


### **Average FPS values of each detector**

In [0]:
print("The average FPS of each detector are:")
print("Darknet: %.2f"%avg_darknet)
print("Python: %.2f"%avg_python)

# **Copying the output files to google drive**

In [0]:
#Copy videos with inference to personal google drives
!cp ./darknet/darknet_video.mp4 /content/gdrive/My\ Drive/DSP-Project/Demo-Videos/darknet_video.mp4
!cp ./pytorch-yolo-v3/output.mp4 /content/gdrive/My\ Drive/DSP-Project/Demo-Videos/pytorch_video.mp4

# **Sources**
* YOLO original website and [paper](https://arxiv.org/abs/1804.02767) by [Joseph Redmon](https://pjreddie.com/darknet/).
* [Dave Ibanyez's blogpost](http://blog.ibanyez.info/blogs/coding/20190410-run-a-google-colab-notebook-to-train-yolov3-using-darknet-in/) on compiling Darknet on Colab.
* [AlexeyAB's](https://github.com/AlexeyAB/darknet) darknet implementation.
* Ayoosh Kathuria's Repository- [A PyTorch implementation of the YOLO v3 object detection algorithm.](https://github.com/ayooshkathuria/pytorch-yolo-v3)